In [203]:
import pandas as pd
import numpy as np
from scipy.stats import zscore


In [151]:
ML = pd.read_excel("123.xlsx")

In [152]:
def scaled_sigmoid(x, k):
    return np.tanh(x * np.arctanh(0.99) / k)

In [221]:
def Preprocessing(ML, rank_list = [30,25,20,15,10,5,-25,-30,-35]):
    # merging with team info
    team_names = pd.read_excel("Team_names.xlsx")
    ML = pd.merge(ML, team_names, on = "Player", how = "left")
    
    # compiling team information & merging
    ML_team = pd.DataFrame()
    ML_team['Team'] = ML['Team'].unique()
    team_total = ML.groupby("Team")["Score"].sum().reset_index().rename(columns={"Score": "Total"})
    ML_team = pd.merge(ML_team, team_total, on = "Team", how = "left")
    ML_team['Average'] = ML_team['Total']/4
    ML_team['Rank'] = ML_team["Total"].rank(ascending = False)
    rank = pd.DataFrame()
    rank['Rank'] = list(range(1,10))
    rank['RankScore'] = rank_list
    ML_team = pd.merge(ML_team, rank, on = "Rank", how = "left")
    ML = pd.merge(ML, ML_team, on = "Team", how = "left")

    # calculating independent rank & other stuff
    indep_rank = []
    indep_df = pd.DataFrame()
    for i in range(ML.shape[0]):
        team_scores = ML_team[['Team', 'Total']]
        score = pd.DataFrame({"Team": "New", "Total": [ML['Total'].iloc[i] - ML['Score'].iloc[i]]})
        team_scores = pd.concat([team_scores, score])
        a = team_scores["Total"].rank(ascending = False).iloc[team_scores.shape[0]-1]
        indep_rank.append(a)
    indep_df["Indep_Rank"] = indep_rank
    ML = pd.merge(ML, indep_df, how = "left", left_index = True, right_index=True)
    ML['Rank_Adjusted'] = ML['Indep_Rank'] - (ML['Score']>0)
    ML['Difference']= ML['Score'] - ML['Average']
    ML_team_sliced = ML_team[['Rank', 'RankScore']].rename(columns={"RankScore": "RankScore_Adjusted", "Rank": "Rank_Adjusted"})
    ML = pd.merge(ML, ML_team_sliced, on = "Rank_Adjusted", how = "left")
    ML.drop(columns = ['Total', 'Rank', 'Average', 'Indep_Rank', 'Rank_Adjusted'], inplace=True)

    return ML

def HAScore_linear(ML):
    ML = Preprocessing(ML)
    ML['HAscore'] = ML['Score']/3 + ML['Difference']/5+ 1.5*ML['RankScore'] - ML['RankScore_Adjusted']
    ML = ML.sort_values(by = "HAscore", ascending = False)
    return ML

def HAScore_sigmoid(ML):
    score_sigmoid = [1, 0.8, 0.7, 0.6, 0.5, 0.4, -0.6, -0.8, -1]
    ML = Preprocessing(ML, score_sigmoid)
    ML['S_sig'] = ML['Score'].apply(lambda x: scaled_sigmoid(x, ML['Score'].max()))
    ML['D_sig'] = ML['Difference'].apply(lambda x: scaled_sigmoid(x, ML['Difference'].max()))
    ML['HAscore'] = ML['S_sig'] + 1.2*ML['RankScore']+ ML['D_sig'] - ML['RankScore_Adjusted']    
    ML = ML.sort_values(by = "HAscore", ascending = False)
    return ML

def HAScore_zscore(ML):
    score_z = [1.2, 1, 0.8, 0.6, 0.4, 0.2, -1, -1.4, -1.8]
    ML = Preprocessing(ML, score_z)
    ML['S_Z'] = zscore(ML['Score'])
    ML['D_Z'] = zscore(ML['Difference'])
    ML['HAscore'] = ML['S_Z'] + 1.2*ML['RankScore']+ ML['D_Z'] - ML['RankScore_Adjusted']    
    ML = ML.sort_values(by = "HAscore", ascending = False)
    return ML

def HAScore_mixed(ML):
    score_z = [1.2, 1, 0.8, 0.6, 0.4, 0.2, -1, -1.4, -1.8]
    ML = Preprocessing(ML, score_z)
    ML['S_Z'] = zscore(ML['Score'])
    ML['D_sig'] = ML['Difference'].apply(lambda x: scaled_sigmoid(x, ML['Difference'].max()))
    ML['HAscore'] = ML['S_Z'] + 1.2*ML['RankScore']+ ML['D_sig'] - ML['RankScore_Adjusted']    
    ML = ML.sort_values(by = "HAscore", ascending = False)
    return ML

In [222]:
linear = HAScore_linear(ML)
sigmoid = HAScore_sigmoid(ML)
z_score = HAScore_zscore(ML)
mixed = HAScore_mixed(ML)

In [223]:
mixed

,Player,Score,Team,RankScore,Difference,RankScore_Adjusted,S_Z,D_sig,HAscore
2,勝又健志,404.2,火山,0.4,408.350,-1.0,2.127053,0.990000,4.597053
1,鈴木優,437.7,海盗,1.2,215.800,1.2,2.303342,0.885064,3.428407
0,瑞原明奈,378.4,海盗,1.2,156.500,1.2,1.991283,0.767546,2.998830
3,伊達朱里紗,215.3,格斗,0.6,203.575,0.2,1.132990,0.866644,2.519634
7,岡田紗佳,212.0,樱花,0.8,150.200,0.6,1.115624,0.750237,2.225861
5,鈴木たろう,211.2,赤坂,1.0,124.875,0.8,1.111414,0.669235,2.180649
6,園田賢,205.1,赤坂,1.0,118.775,0.8,1.079313,0.646825,2.126138
4,仲林圭,266.5,海盗,1.2,44.600,1.2,1.402423,0.281276,1.923700
8,渋川難波,147.2,樱花,0.8,85.400,0.8,0.774622,0.503143,1.437765
10,松本吉弘,100.0,AB,0.2,105.525,0.2,0.526238,0.594077,1.160314


In [ ]:
\